# An analysis of the Chinook music store database
## Answering business questions with SQL

This project looks into the SQLite database of a fictional music streaming store called Chinook. The goal is to gain insights that can guide future decision making.<br>
<br>
Specifically, I will use the data to make decisions in four scenarios:
- We have been offered several albums of different genres, but our budget only allows us to purchase some of them. Which one should we buy based on past sales data? 
- We are introducing a new bonus programm for the best performing sales rep. Who should receive the bonus? 
- We decide to reduce our costs by focussing on the countries that we are most succesful in. I will use the database to determine which countries we should focus on.
- Lastly, we need to decide whether we should focus on offering whole albums or single tracks. I will use our data to compare order volumes for entire albums and single tracks. 

### Exploring the data

<img src="chinook_schema.png" width="600"/>

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Which albums should we purchase based on past sales data?

We have been offered four albums by different artists in different genres to sell in our US store:

- Regal	(Hip-Hop)
- Red Tone (Punk)
- Meteor and the Girls (Pop)
- Slim Jim Bites (Blues)

Our budget allows us to purchase three. All of the artists would be new to our store, so we don't know anything about their popularity amongst our US customers. To decide which ones to buy we will instead look into the popularity of each genre amonst our US customers. 

In [3]:
%%sql
WITH track_genre AS
        (
        SELECT
            t.track_id,
            g.name 'genre'
        FROM track t
        INNER JOIN genre g ON t.genre_id = g.genre_id
        ),
    invoice_track_genre AS
        (
        SELECT
            il.invoice_id,
            tg.track_id,
            tg.genre
        FROM invoice_line il
        INNER JOIN track_genre tg ON il.track_id = tg.track_id
        ),
    invoice_country AS
        (
        SELECT
            i.invoice_id,
            c.country
        FROM customer c
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        ),
    genres_usa AS
        (
        SELECT
            itg.genre,
            COUNT(itg.genre) 'absolute'
        FROM invoice_track_genre itg
        LEFT JOIN invoice_country ic ON itg.invoice_id = ic.invoice_id
        GROUP BY itg.genre, ic.country
        HAVING ic.country = 'USA'
        ),
    total_usa AS
        (
        SELECT TOTAL(absolute) 'total'
        FROM genres_usa
        )

SELECT
    gu.*,
    round(absolute / ta.total * 100, 2) "percentage"
FROM genres_usa gu
LEFT JOIN total_usa ta
ORDER BY 3 DESC

 * sqlite:///chinook.db
Done.


genre,absolute,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on this analysis, the store should purchase albums from the artists Red Tone (Punk), Slim Jim Bites (Blues), Regal (Hip-Hop). 

# Who is our best performing sales rep?

In [4]:
%%sql

WITH 
     customer_total AS
        (
        SELECT
            c.support_rep_id,
            c.customer_id,
            SUM(i.total) 'total'
        FROM customer c
        INNER JOIN invoice i on i.customer_id = c.customer_id
        GROUP BY c.customer_id
        ),
    rep_total AS
        (
        SELECT
             SUM(total) 'total',
             support_rep_id
         FROM customer_total
         GROUP BY 2
        )

SELECT
    e.employee_id, 
    e.first_name || " " || e.last_name 'name',
    ROUND(rt.total, 2)'total_sales USD'
FROM employee e
INNER JOIN rep_total rt on rt.support_rep_id = e.employee_id

 * sqlite:///chinook.db
Done.


employee_id,name,total_sales USD
3,Jane Peacock,1731.51
4,Margaret Park,1584.0
5,Steve Johnson,1393.92


Based on this summary table, our sales rep Jane Peackock has shown the highest volume of sales so far. It should be her who receives the first award.

# Comparing our customer base and sales across different countries

To determine in which countries we are doing well, we will compare the following data by country:
- total number of customers
- total value of sales
- average value of sales/customer
- average order value.

In [5]:
%%sql
WITH
    customer_orders AS
        (
        SELECT
            CASE
                WHEN
                (
                    SELECT count(*)
                    FROM customer 
                    WHERE country = c.country
                ) = 1 THEN "Other"
                ELSE c.country
            END AS 'country',
            SUM(i.total) 'value',
            COUNT(i.invoice_id) 'orders'
        FROM customer c
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        GROUP BY c.customer_id
        ),
    country_sales AS
        (
        SELECT
            country,
            COUNT(country) 'customers',
            ROUND(SUM(value), 2) 'sales_volume',
            ROUND((SUM(value) / SUM(orders)), 2) 'average_order'
        FROM customer_orders
        GROUP BY country
        )

SELECT
    country, 
    customers, 
    sales_volume,
    average_order,
    ROUND((sales_volume / customers), 2) 'avg_lifetime_value'
FROM
    (SELECT 
        *,
        CASE
             WHEN country = 'Other' THEN 1
             ELSE 0
        END 'sort'
     FROM country_sales cs
    )
ORDER BY sort ASC, sales_volume DESC

 * sqlite:///chinook.db
Done.


country,customers,sales_volume,average_order,avg_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0


# Comparing purchases of whole albums vs. single tracks

Currently, out store purchases only entire albums from labels. Customers can then buy any track from an album, or they can buy the whole album by adding all the tracks from an album to their order. To reduce our expenses, we are considering buying only select tracks instead of entire albums. This might drive away customers looking to buy an entire album. To get an idea of the impact such a change could have, we will see how many of our orders actually contain an entire album. 

In [6]:
%%sql 

WITH invoice_line_album AS
    (
    SELECT
        il.invoice_id,
        il.track_id 'track_ila',
        t.album_id 'album_ila'
    FROM invoice_line il
    LEFT JOIN track t
    ON t.track_id = il.track_id
    ),
    whole_album AS
    (
    SELECT
        invoice_id, 
        CASE
            WHEN 
                (
                    SELECT track_ila 
                    FROM invoice_line_album ila2
                    WHERE invoice_id = ila.invoice_id
                    EXCEPT
                    SELECT track_id
                    FROM track
                    WHERE album_id = ila.album_ila
                ) IS NULL
                AND 
                (
                    SELECT track_id
                    FROM track
                    WHERE album_id = ila.album_ila
                    EXCEPT
                    SELECT track_ila 
                    FROM invoice_line_album ila2
                    WHERE invoice_id = ila.invoice_id
                ) IS NULL
                THEN 1
            ELSE 0
        END 'whole_album'
    FROM invoice_line_album ila
    GROUP BY 1
    )

SELECT
    COUNT(*) 'total_orders',
    SUM(whole_album) 'whole_album_purchases',
    ROUND((CAST(SUM(whole_album) AS float) / CAST(COUNT(*) AS FLOAT)), 2) 'whole_album_orders'
FROM whole_album

 * sqlite:///chinook.db
Done.


total_orders,whole_album_purchases,whole_album_orders
614,114,0.19


Our analysis shows that about 19% of our orders are purchases of an entire album. This would suggest that switching to offering only select songs could be a good idea.